In [2]:
from path_arquivos import *
import pandas as pd
import numpy as np

In [3]:
df_28 = pd.read_excel(ar_xlsx.ar_28, usecols= ['NUMOS','CODPROD', 'DESCRICAO', 'ENDERECO_ORIG', 'RUA', 'PREDIO', 'NIVEL', 'APTO','RUA_1', 'PREDIO_1', 'NIVEL_1', 'APTO_1', 'QT', 'POSICAO', 'CODFUNCOS', 'CODFUNCESTORNO', 'Tipo O.S.', 'FUNCOSFIM','FUNCGER'])
df_28 = df_28.loc[(df_28['NIVEL'].between(2,8)) & (df_28['Tipo O.S.'] == '58 - Transferencia de Para Vertical')]
df_28[['CODFUNCESTORNO', 'CODFUNCOS']] = df_28[['CODFUNCESTORNO', 'CODFUNCOS']].fillna(0).astype(int)
df_28['ID_COD'] = df_28['ENDERECO_ORIG'].astype(str) + " - " + df_28['CODPROD'].astype(str)

df_aereo = pd.read_csv(ar_csv.ar_end, header= None, names= col_name.cEnd)
df_aereo = df_aereo.loc[df_aereo['TIPO_PK'] == 'AE']
df_aereo = df_aereo[['COD_END', 'COD', 'QTDE', 'ENTRADA', 'SAIDA']]
df_aereo['ID_COD'] = df_aereo['COD_END'].astype(str) + " - " + df_aereo['COD'].astype(str)

df_func = pd.read_excel(ar_xlsx.ar_func, sheet_name= 'FUNCIONARIOS', usecols= ['MATRICULA', 'TIPO'])

df = df_28.merge(df_aereo, left_on= 'ID_COD', right_on= 'ID_COD', how= 'left').infer_objects(copy=False).fillna(0).drop(['COD_END', 'COD'],axis= 1)
df = df.merge(df_func, left_on= 'CODFUNCOS', right_on= 'MATRICULA', how= 'left').drop(columns=['MATRICULA'])
df['TIPO'] = df['TIPO'].fillna("FUNC")   

c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:

df['QTDE'] = df['QTDE'].astype(str).str.replace('.', '', regex=False)
df['QT'] = df['QT'].astype(str).str.replace('.', '', regex=False)

df['QTDE'] = df['QTDE'].astype(str).str.replace(',', '.', regex=False)
df['QT'] = df['QT'].astype(str).str.replace(',', '.', regex=False)

df['TIPO_OP'] = np.where(df['QTDE'].astype(float) >= df['QT'].astype(float), "baixa", "retorno")

df_fim = df.loc[(df['TIPO'] != 'EMPILHADOR') & (df['TIPO'] != 'ABASTECEDOR') & (df['CODFUNCOS'] > 0) & (df['CODFUNCESTORNO']== 0) & (df['QTDE'].astype(float) == 0)]
df_fim = df_fim[['NUMOS','CODPROD', 'DESCRICAO', 'RUA', 'PREDIO', 'NIVEL','APTO', 'RUA_1', 'PREDIO_1', 'NIVEL_1', 'APTO_1','FUNCOSFIM','TIPO_OP','QTDE','QT']].copy()
df_fim = df_fim.sort_values(by=['RUA', 'PREDIO'], ascending= True, axis= 0)    

In [5]:
display(df_fim)

,NUMOS,CODPROD,DESCRICAO,RUA,PREDIO,NIVEL,APTO,RUA_1,PREDIO_1,NIVEL_1,APTO_1,FUNCOSFIM,TIPO_OP,QTDE,QT
928,47174211,474929,BISC CORY 80GR RECH HIPOPO DOCE DE LEITE,2,41,5,501,2,81,1,104,LUCIANO ALBUQUERQUE RIPARDO,retorno,0,20
143,47170702,473001,"CHOC NESTLE KIT KAT 185,6GR CELEBREAK",6,1,2,201,11,56,1,101,CLEBER VIEIRA DA SILVA,retorno,0,180
1013,47175054,458736,AMAC DOWNY 900ML CONC PAIXAO,26,49,5,502,26,42,1,101,LUCIANO ALBUQUERQUE RIPARDO,retorno,0,108
81,47168187,475378,SABAO LIQ OMO CICLO RAP 750ML AC CUIDADO,27,2,4,402,26,46,1,202,JOAO ALLISON DE LIRA MONTEIRO,retorno,0,384
1012,47175052,456038,SABAO LIQ BRILHANTE 3LT LIMPEZA TOTAL,27,27,3,302,22,23,1,101,LUCIANO ALBUQUERQUE RIPARDO,retorno,0,15
586,47171561,455529,LAPIS COR F CAST 24 COR,28,4,2,202,28,38,1,102,JOAO ALLISON DE LIRA MONTEIRO,retorno,0,80
1367,47180951,465388,CHOC FERRERO KINDER OVO 2X20GR ROSA T2,200,1,4,402,1,17,3,302,LUCIANE RIBEIRO DA SILVA,retorno,0,2400
